In [9]:
#SETTING UP
import credentials
import settings
import mysql.connector
import pandas as pd
import time
import itertools
import math

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly.express as px
import datetime
from IPython.display import clear_output

from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import nltk

#LOAD DATA FROM MYSQL
db_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd=credentials.MYSQLPASSWORD,
    database="twitterdb",
    auth_plugin='mysql_native_password',
    charset = 'utf8'
)


In [ ]:
#PLOT SHOWING LATEST TWITTER DATA WITHIN 30 MINUTES THAT UPDATES AUTOMATICALLY.

######################
##  POLARITY LEGEND ##
##   -1 NEGATIVE    ##
##    0 NEUTRAL     ##
##   +1 POSITIVE    ##
######################

while True:
    clear_output()
    db_connection = mysql.connector.connect(
        host="localhost",
        user="root",
        passwd=credentials.MYSQLPASSWORD,
        database="TwitterDB",
        charset = 'utf8'
    )

    timenow=datetime.datetime.now().strftime("%H:%M:%S")

    query = "SELECT id_str, text, created_at, polarity FROM {} WHERE created_at >= '{}' " \
                            .format(settings.TABLE_NAME, timenow)
    df = pd.read_sql(query, con=db_connection)
    df['created_at']=pd.to_datetime(df['created_at'])

        #CLEAN AND TRANSFORM DATA TO ENABLE TIME SERIES
    result=df.groupby([pd.Grouper(key='created_at', freq='2s'),'polarity']).count().unstack(fill_value=0).stack().reset_index()

    result = result.rename(columns={"id_str": "Num of '{}' mentions".format('Ethereum'[0]), "created_at":"Time in ET"})

    fig=px.line(result,x='Time in ET',
                    y="Num of '{}' mentions".format("Ethereum"),color='polarity')
    fig.show()
    time.sleep(60)

